<a href="https://colab.research.google.com/github/liorhirsch/Kaggle-Landmark-ML-Challange/blob/master/download-save-photos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!jupyter nbextension enable hinterland/hinterland
!pip install jupyter_contrib_nbextensions
!jupyter contrib nbextension install --user
!jupyter nbextension enable varInspector/main

Enabling notebook extension hinterland/hinterland...
      - Validating: problems found:
        - require?  X hinterland/hinterland
    100% |████████████████████████████████| 20.9MB 1.8MB/s 
    100% |████████████████████████████████| 870kB 20.8MB/s 
    100% |████████████████████████████████| 471kB 22.3MB/s 
  Running setup.py bdist_wheel for jupyter-latex-envs ... - \ done
  Stored in directory: /root/.cache/pip/wheels/0d/71/2a/164491997299b9f2479a251e254323fe35d946779e18f27956
Successfully built jupyter-latex-envs
[I 16:58:43 InstallContribNbextensionsApp] jupyter contrib nbextension install --user
[I 16:58:43 InstallContribNbextensionsApp] Installing jupyter_contrib_nbextensions nbextension files to jupyter data directory
[I 16:58:44 InstallContribNbextensionsApp] Installing /usr/local/lib/python3.6/dist-packages/jupyter_contrib_nbextensions/nbextensions/limit_output -> limit_output
[I 16:58:44 InstallContribNbextensionsApp] Making directory: /root/.local/share/jupyter/nbexte

Download the dataset from kaggle : 
---

**Upload your kaggle.json** file here when activating this bulk



In [0]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d google/google-landmarks-dataset
! mkdir Google-Landmarks-Dataset
! unzip google-landmarks-dataset.zip -d Google-Landmarks-Dataset

Saving kaggle.json to kaggle.json
 98% 104M/106M [00:02<00:00, 27.7MB/s] 
100% 106M/106M [00:02<00:00, 37.5MB/s]
mkdir: cannot create directory ‘Google-Landmarks-Dataset’: File exists
Archive:  google-landmarks-dataset.zip
  inflating: Google-Landmarks-Dataset/test.csv  
  inflating: Google-Landmarks-Dataset/train.csv  
  inflating: Google-Landmarks-Dataset/index.csv  


Old way : 
---



First, we install kaggle to use its api via command

In [0]:
!pip install kaggle

**Authorize with kaggle's api key and token** 

We're doing it in order to download the dataset.
we insert the credentials and it generates kaggle.json in /root/.kaggle.

**NOTE** : The credentials should be inserted dynamically and not saved here in the public notebook because it should be secured!!!!

In [0]:
import os
from getpass import getpass
user = getpass('Kaggle Username: ')
key = getpass('Kaggle API key: ')

if '.kaggle' not in os.listdir('/root'):
    !mkdir ~/.kaggle
!touch /root/.kaggle/kaggle.json
!chmod 666 /root/.kaggle/kaggle.json
with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write('{"username":"%s","key":"%s"}' % (user, key))
!chmod 600 /root/.kaggle/kaggle.json

Kaggle Username: ··········
Kaggle API key: ··········


In [0]:
!kaggle datasets download -d google/google-landmarks-dataset

100% 106M/106M [00:01<00:00, 58.9MB/s] 
100% 106M/106M [00:01<00:00, 68.9MB/s]


**Unzips the dataset to new directory**

In [0]:
! mkdir google-landmarks-dataset
! unzip google-landmarks-dataset.zip -d google-landmarks-dataset

Archive:  google-landmarks-dataset.zip
  inflating: google-landmarks-dataset/test.csv  
  inflating: google-landmarks-dataset/train.csv  
  inflating: google-landmarks-dataset/index.csv  


# Get 100 landmarks
---



In [0]:
# import tensorflow as tf
import pandas as pd
import random
import os
import requests
import shutil

# os.system("qsub -I -V -N kukk -l nodes=1:gpus=8:V100")

_TRAIN_FOLDER_ = "train/"


def getRandomClasses(file="None", total=100):
    if file is None:
        return []

    print("Loading " + file + "...")
    train = pd.read_csv(file, skiprows=1, header=None, names=["#", "Image", "Class"])
    
    print("Removing empty lines ...")
    train.dropna()

    print("Getting unique classes")
    classes = train['Class'].unique().tolist()

    print("Shuffling classes...")
    random.shuffle(classes)

    print("Grouping the classes")
    groups = train.groupby('Class').size().reset_index(name='counts');
    
    
    print("Taking classes with more than 1000 images and less than 8000")
    classesWithMoreThanThousandPhotos = groups[(groups.counts >= 1000) & (groups.counts <= 8000)];
    classes = classesWithMoreThanThousandPhotos["Class"].tolist();
    
    print(len(classes))
    
    print("Scanning classes...")
    list = []
    for _class in classes:
        images = train.loc[train['Class'] == _class]
        if _class == 'None':
            continue

        if images.shape[0] < total:
            continue

        list.append(_class)

        if len(list) >= total:
            break

    return train, list[:total]
  
def createFolder(folder = None):
    if folder is None:
        return

    if not os.path.exists(folder):
        os.makedirs(folder)

def createTrainingCsvData(folder = None):
    if folder is None:
        print("No training data supplied")
        return

    # Create training folder
    createFolder(folder)

    # Generate 100 classes out of the landmark database
    train, classes = getRandomClasses('C:/Projects/ML/train.csv')

    print("Found "+str(len(classes)) + " classes")

    # generate csv files with images
    for _class in classes:
        with open(_TRAIN_FOLDER_ + _class + ".csv", 'a+') as f:
            print("Exporting " + _class + "...")
            images = train.loc[train['Class'] == _class]
            images.to_csv(_TRAIN_FOLDER_ + _class + ".csv", header=False, index=False)

    print("landmark csv data generated successfully.")

def fetch_image(path,folder, name):
    url = path

    try:
        response = requests.get(url, stream=True)
        with open(folder + name + '.jpg', 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
        del response
    except OSError:
        print("Can't download " + path)

def downloadTrainingImages(folder = None):
    if folder is None:
        print("No training data supplied")
        return

    if not os.path.exists(folder):
        print("Training data folder does not exists")
        return

    if len(os.listdir(folder)) == 0:
        print("Training folder is empty")
        return
    else:
        files = []
        for fname in os.listdir(folder):
            path = os.path.join(folder, fname)
            if os.path.isdir(path) or fname == ".DS_Store":
                # skip directories
                continue
            files.append(fname)

        counter = 1
        for file in files:
            new_folder = folder + file.replace(".csv", '')
            print("[" + str(counter) + "/" + str(len(files)) + "] Creating " + new_folder + " folder...")
            createFolder(new_folder)

            print("[" + str(counter) + "/" + str(len(files)) + "] Loading " + file + "...")
            images = pd.read_csv(folder + file, names=["#", "Image", "Class"])
            images.drop(['Class'], axis=1, inplace=True)
            # images = images['Image'].tolist()

            for index, img in images.iterrows():
                print("[" + str(counter) + "/" + str(len(files)) + "] - [" + str(index+1) + "/" + str(images.shape[0]) + "] Downloading " + img["Image"] + "...")
                fetch_image(img["Image"], new_folder+"/", img['#'])
                if (index+1) > 100:
                    break;

            counter = counter + 1
if __name__ == '__main__':
    createTrainingCsvData(_TRAIN_FOLDER_)
    # downloadTrainingImages(_TRAIN_FOLDER_)



In [0]:
import time
start_time = time.time();
print("start fetching images at %s" %start_time)

downloadTrainingImages(_TRAIN_FOLDER_)

print("--- %s seconds ---" % (time.time() - start_time))


start fetching images at 1544736146.6206937
[1/100] Creating train/2743 folder...
[1/100] Loading 2743.csv...
[1/100] - [1/8922] Downloading https://lh4.googleusercontent.com/-AVEfPLKUjwQ/SdF7VIFGoHI/AAAAAAAABNg/O7-q8jnIuvc/s1600/...
[1/100] - [2/8922] Downloading https://lh4.googleusercontent.com/-6F2ixNiR2G8/SVAPg9kYJuI/AAAAAAAABLY/DDqc4g6zjsM/s1600/...
[1/100] - [3/8922] Downloading https://lh3.googleusercontent.com/-qHUU6VMq6kA/UuJG3rquB6I/AAAAAAABjb4/c9VQZvCe6gs/s1600/...
[1/100] - [4/8922] Downloading https://lh6.googleusercontent.com/-C3ZKusAqRps/S4fIIvlc86I/AAAAAAAACcg/Cl4cVU6aGl0/s1600/...
[1/100] - [5/8922] Downloading https://lh4.googleusercontent.com/-pa6emLCDQGQ/SuXbuwk3c8I/AAAAAAAADmE/rsPAeA9ReII/s1600/...
[1/100] - [6/8922] Downloading https://lh4.googleusercontent.com/--asAFt9saKU/T__jYO8YnUI/AAAAAAAAAcw/epUPoKe9CCE/s1600/...
[1/100] - [7/8922] Downloading https://lh5.googleusercontent.com/-AEvOW1bNze8/Rcvx0BpVp4I/AAAAAAAAA4E/mbWocWG_wJw/s1600/...
[1/100] - [8/8922] Dow

# Upload data from google drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import zipfile
with zipfile.ZipFile("drive/My Drive/content/all.zip", 'r') as zip_ref:
    zip_ref.extractall("photos")

# Build training data

In [0]:

import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from joblib import Parallel, delayed
import multiprocessing

In [0]:
DATADIR = "photos"
CATEGORIES =  os.listdir(DATADIR);
training_data = []
IMG_SIZE = 100
   
def create_training_data(category):
    data = []
    print("Building training data for landmark id %s" %category);
    path = os.path.join(DATADIR, category)  # path tolandmark photos
    landmark_id = category
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            data.append([new_array, landmark_id])
        except Exception as e:
            pass   # do nothing
    return data;
          
num_cores = multiprocessing.cpu_count();
training_data_nested=Parallel(n_jobs=num_cores)(delayed(create_training_data)(category) for category in CATEGORIES)
# training_data=[filename for path in dirs for filename in os.listdir(path)]


In [0]:
print(training_data_nested[0])

[[array([[183, 209,  74, ..., 182, 180, 201],
       [198, 209,  73, ..., 178, 190, 188],
       [199, 207,  69, ..., 181, 177, 178],
       ...,
       [148, 130, 147, ..., 151, 141, 132],
       [160, 162, 159, ...,  99, 105, 105],
       [170, 175, 163, ..., 101,  99, 105]], dtype=uint8), '7661'], [array([[ 88, 188, 194, ...,  55,  48,  54],
       [ 63,  93, 151, ...,  47,  55,  52],
       [ 44,  50,  74, ...,  56,  34,  39],
       ...,
       [214, 202, 208, ..., 203, 201, 206],
       [207, 210, 195, ..., 213, 204, 209],
       [208, 210, 207, ..., 206, 219, 212]], dtype=uint8), '7661'], [array([[ 13,  13,  16, ..., 186, 133, 143],
       [ 16,  14,  22, ..., 174, 152, 204],
       [107,  23,  34, ...,  79, 102, 134],
       ...,
       [164, 138,  88, ..., 103, 103, 129],
       [ 74,  45,  39, ...,  57, 138,  99],
       [ 33,  45,  61, ...,  75,  43,  91]], dtype=uint8), '7661'], [array([[ 65, 102, 122, ..., 129, 128, 125],
       [ 58,  75, 123, ..., 132, 128, 124],
       

In [0]:
training_data=[landmark_id_pair for landmark_photos in training_data_nested for landmark_id_pair in landmark_photos]

In [0]:
print(training_data[0])

[array([[183, 209,  74, ..., 182, 180, 201],
       [198, 209,  73, ..., 178, 190, 188],
       [199, 207,  69, ..., 181, 177, 178],
       ...,
       [148, 130, 147, ..., 151, 141, 132],
       [160, 162, 159, ...,  99, 105, 105],
       [170, 175, 163, ..., 101,  99, 105]], dtype=uint8), '7661']


In [0]:
import random

random.shuffle(training_data)

In [0]:
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

# We need to convert the List of images to a numpy array for Tensorflow
# We create a numpy array where the image is in the shape of (50,50,1). 1 since this is a gray scale
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
print(X[0].shape)

(100, 100, 1)


# Train

In [0]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Conv3D
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

In [0]:
X = X/255.0


In [0]:
print(X.shape[1:])

(100, 100, 1)


In [0]:
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

# #Flatening the data
model.add(Flatten())
model.add(Dense(120))
model.add(Activation("relu"))


In [0]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 64)        640       
_________________________________________________________________
activation (Activation)      (None, 98, 98, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 64)        36928     
_________________________________________________________________
activation_1 (Activation)    (None, 47, 47, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 33856)             0         
__________

In [0]:
adamOptimizer = tf.keras.optimizers.Adam(lr=0.001)

model.compile(loss = 'sparse_categorical_crossentropy', 
             optimizer='adam',
             metrics=['accuracy'])

In [0]:
print (X.shape)

(10055, 100, 100, 1)


In [0]:

model.fit(X, y, batch_size=32, epochs = 3, validation_split=0.1)

AttributeError: ignored

In [0]:
train_loss, train_acc = model.evaluate(X,y)
